## **1. Install and import libraries**

In [ ]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0

In [ ]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# **2. Set up model ID and quantization config**

In [ ]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

# Sử dụng kỹ thuật quantile lượng tự hóa giảm kích thước mô hình
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


# **3. Install and load model**

In [ ]:
# Load tham số mode vào và tokenizer
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

# **4. Chat with model**

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
promt = """
một cộng một bằng bao nhiêu
"""
model_inputs = tokenizer(promt, return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs)[0]
answer = tokenizer.decode(generated_ids, skip_special_tokens=True)
answer.split('\n\n')[1]

'1. Đưa ra một số phương trình tương đối với các đại lượng tương đối với nhau.\n2. Xác định một đại lượng tương đối với một đại lượng khác.\n3. Xác định một đại lượng tương đối với một đại lượng khác dưới một đại lượng chung.\n4. Xác định một đại lượng tương đối với một đại lượng khác dưới một đại lượng chung và so sánh với một đại lượng khác.\n5. Xác định một đại lượng tương đối với một đại lượng khác dưới một đại lượng chung và so sánh với một đại lượng khác dưới một đại lượng khác.\n6. Xác định một đại lượng tương đối với một đại lượng khác dưới một đại lượng chung và so sánh với một đại lượng khác dưới một đại lượng khác dưới một đại lượng khác.\n7. Xác định một đại lượng tương đối với một đại lượng khác dưới một đại lượng chung và so sánh với một đại lượng khác dưới một đại lượng khác dưới một đại lượng khác.\n8. Xác định một đại lượng tương đối với một đại lượng khác dưới một đại lượng chung và so sánh với một đại lượng khác dưới một đại lượng khác dưới một đại lượng khác.\n9. Xá

In [ ]:
answer

'\nhãy kể cho tôi truyện tấm cám\n\nVào một ngày nọ, tấm cám đang đứng trên một cây cổ thụ, tập trung vào việc tìm kiếm đường đi để trở về nhà. Bởi vì tấm cám đã bị mất trên tất cả các đường đi trước đó, nó đã quyết định tìm kiếm một cách mới để trở về nhà.\n\nTấm cám bắt đầu đi theo một con đường mới, nhưng nó đã nhanh chóng thấy rằng nó không đi đến nhà. Tấm cám bỏ lại con đường đó và bắt đầu tìm kiếm một cách khác.\n\nNó đi theo một con đường khác, nhưng lại không đi đến nhà. Tấm cám bỏ lại con đường đó và bắt đầu tìm kiếm một cách khác.\n\nTấm cám đi theo một con đường khác nữa, nhưng lại không đi đến nhà. Tấm cám bỏ lại con đường đó và bắt đầu tìm kiếm một cách khác.\n\nNhưng lần này, tấm cám đã tìm thấy một con đường mới. Nó đi theo con đường đó và cuối cùng đã trở về nhà. Tấm cám hạnh phúc vì nó đã tìm thấy cách đi đến nhà và quay trở lại với gia đình mình.'